In [1]:
"""
parallel MRI reconstruction using an online paradigm.

requires PySAP and PySAP-MRI, and their dependencies.

"""
import os
import time
import numpy as np
import scipy.fftpack as pfft
import matplotlib.pyplot as plt
from skimage.measure import label
from scipy.ndimage.morphology import binary_closing

In [2]:
from modopt.opt.linear import Identity
from modopt.opt.algorithms import Condat
from modopt.math.metrics import ssim

In [3]:
from mri.operators.fourier.non_cartesian import NUFFT
from mri.operators import OWL
from parallel_mri_online.gradient import Grad2D_pMRI
from parallel_mri_online.linear import Pywavelet2

/home/pac/.pyenv/versions/mri/lib/python3.9/site-packages/pysap_mri-0.2.2-py3.9.egg/mri/operators/fourier/cartesian.py:34: UserWarning: pynufft python package has not been found. If needed use the master release. Till then you cannot use NUFFT on GPU
/home/pac/.pyenv/versions/mri/lib/python3.9/site-packages/pysap_mri-0.2.2-py3.9.egg/mri/operators/fourier/non_cartesian.py:42: UserWarning: gpuNUFFT python package has not been found. If needed please check on how to install in README


In [4]:
# Loading input data
folder = '/home/pac/gits/phd/online_2D/multi_channel_non_cartesian/2017-09-18_wilson_complete/'
data_path = folder + 'meas_MID111_CSGRE_ref_N512_FID24209.npy'
folder += 'sparkling_nc34x3072/'
kspace_ref = np.load(data_path)
Il = pfft.ifftshift(pfft.ifft2(pfft.fftshift(kspace_ref), axes=[1, 2])).astype("complex128")
print(Il.shape)
I = np.sqrt(np.sum(np.abs(Il) ** 2, axis=0))
samples = np.load(
    folder + "GA_GradientFile_Samples_SPARKLING_N512_R3_nc34x3073_Dt10us_densitytrick_BB_upsampled_Smax302.npy")
samples = np.reshape(samples, (34 * 3072, 2))
im_mask = np.zeros(I.shape)
im_mask[np.abs(I) > 3e-08] = 1

(32, 512, 512)


In [5]:
def get_largest_cc(segmentation):
    labels = label(segmentation, background=0)
    bincounts = np.bincount(labels.flat)
    idx = np.argsort(bincounts)[::-1]
    largestCC = np.copy(labels == idx[1])
    return largestCC

In [6]:
im_mask = binary_closing(get_largest_cc(im_mask), np.ones((5, 5)))
b_size = 34
t_it = 4
TR = 0.550
batches = np.arange(b_size, 34 + 1, b_size)  # [34]
print(batches)
if batches.shape[0] > 1:
    nb_it_batches = [np.floor(b_size * 1.0 * TR / t_it).astype('int') for _ in
                     range(batches.shape[0] - 1)]
    nb_it_batches.append(200)
else:
    nb_it_batches = [200]

[34]


In [7]:
print(nb_it_batches)

[200]


In [8]:
directory = folder + 'online_reconstrcution_batch_size_' + str(b_size)

In [9]:
dir_exist = False
try:
    os.stat(directory)
    dir_exist = True
except:
    dir_exist = False
    os.mkdir(directory)
    os.mkdir(directory + '/masks_batches/')
    os.mkdir(directory + '/images_batches/')
    os.mkdir(directory + '/times_batches/')
if dir_exist:
    try:
        os.stat(directory + '/masks_batches/')
    except:
        os.mkdir(directory + '/masks_batches/')
    try:
        os.stat(directory + '/images_batches/')
    except:
        os.mkdir(directory + '/images_batches/')
    try:
        os.stat(directory + '/times_batches/')
    except:
        os.mkdir(directory + '/times_batches/')

In [10]:
# Generate the subsampled kspace
kspace = np.load(folder + 'GA_meas_MID123_CSGRE_N512_sparkling_nc34_OS2_BR1536_FID24221.npy')

In [11]:
prox_op = Identity()
# Get the locations of the kspace samples
linear_op = Pywavelet2("sym8", nb_scale=4, multichannel=True)
coeffs = linear_op.op(Il)
alpha = 1e-05 / 34
beta = 1e-12 / 34
prox_dual_op = OWL(alpha=alpha,
                   beta=beta,
                   bands_shape=linear_op.coeffs_shape,
                   mode='band_based',
                   n_coils=32,
                   n_jobs=1)

[(46, 46), [(46, 46), (46, 46), (46, 46)], [(77, 77), (77, 77), (77, 77)], [(139, 139), (139, 139), (139, 139)], [(263, 263), (263, 263), (263, 263)]]


In [12]:


fourier_op = NUFFT(samples=samples, shape=I.shape, platform='opencl',
                   Kd=(1024, 1024), Jd=(5, 5),
                   n_coils=32, verbosity=0)
# kspace_mask=np.ones((32, 34 * 3072)))

/home/pac/.pyenv/versions/mri/lib/python3.9/site-packages/pyopencl/__init__.py:266: CompilerWarning: Non-empty compiler output encountered. Set the environment variable PYOPENCL_COMPILER_OUTPUT=1 to see more.
  warn("Non-empty compiler output encountered. Set the "


In [ ]:
gradient_op = Grad2D_pMRI(data=np.zeros_like(kspace.reshape((32, 34 * 3072))),
                          fourier_op=fourier_op)

Start the POGM reconstruction
max_iter = 200

In [ ]:
# Define the optimizer
sigma = 0.5
eps = 5e-8
norm = 1.0
tau = 1.0 / (gradient_op.spec_rad / 2 + sigma * norm ** 2 + eps)
opt = Condat(
    x=np.zeros((32, *fourier_op.shape), dtype="complex128"),
    y=np.zeros_like(coeffs),
    grad=gradient_op,
    prox=prox_op,
    prox_dual=prox_dual_op,
    linear=linear_op,
    cost=None,
    rho=1.0,
    sigma=sigma,
    tau=tau,
    rho_update=None,
    sigma_update=None,
    tau_update=None,
    auto_iterate=False)
# Setting tome vector
time_batches = np.asarray([TR * b_size])

In [ ]:
# Setting SSIM vector
final_ssim = np.asarray([0])

In [ ]:
# Setting cost vector
final_cost = np.asarray([np.sum(np.abs(kspace.flatten()) ** 2)])

In [ ]:
for idx, batch in enumerate(batches):
    obs_data = np.copy(opt._grad.obs_data)
    obs_data[:, :batch * 3072] = np.reshape(kspace[:, :batch, :], (32, batch * 3072))
    opt._grad = Grad2D_pMRI(data=obs_data,
                            fourier_op=fourier_op,
                            linear_op=opt._grad.spec_rad)
    opt._grad.fourier_op.kspace_mask[...] = 0
    opt._grad.fourier_op.kspace_mask[:, :batch * 3072] = 1
    cost_func_batch = []
    ssim_batches = []
    opt._prox_dual = OWL(alpha=alpha * batch,
                         beta=beta * batch,
                         bands_shape=linear_op.coeffs_shape,
                         mode='band_based',
                         n_coils=32,
                         n_jobs=-1)
    print(opt._prox_dual.weights[0])
    time_it = []
    for _ in range(nb_it_batches[idx]):
        print(_)
        start = time.time()
        opt._update()
        stop = time.time()
        time_it.append(stop - start)
        cost_func_batch.append(np.sum(np.abs(opt._grad.op(opt._x_new).flatten() -
                                             kspace.flatten()) ** 2) +
                               opt._prox_dual.cost(opt._linear.op(opt._x_new)))
        ssim_batches.append(ssim(np.sqrt(np.sum(np.abs(opt._x_new) ** 2,
                                                axis=0)), I, im_mask))
    time_batches = np.concatenate((time_batches, np.asarray(time_it)))
    np.save(directory + '/times_batches/time_batch_nb_{0:02d}.npy'.format(idx), time_it)

    np.save(directory + '/images_batches/image_batch_nb_{0:02d}.npy'.format(idx), opt._x_new)
    imsave_path = directory + '/images_batches/I_batch_{0:02d}.png'.format(idx)
    plt.imsave(fname=imsave_path,
               arr=np.sqrt(np.sum(np.abs(opt._x_new) ** 2, axis=0)), cmap='gray')

    # Updating cost function
    final_cost = np.concatenate((final_cost, np.asarray(cost_func_batch)))
    final_ssim = np.concatenate((final_ssim, np.asarray(ssim_batches)))

In [ ]:
np.save(directory + '/resume_time.npy', time_batches)
np.save(directory + '/resume_cost.npy', final_cost)
np.save(directory + '/resume_ssim.npy', final_ssim)
plt.figure()
plt.plot(final_cost)
plt.figure()
plt.plot(final_ssim)
plt.figure()
plt.imshow(np.sqrt(np.sum(np.abs(opt._x_new) ** 2, axis=0)))
plt.show()